In [2]:
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from sympy import Point, Polygon 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [158]:
#Feed-ul de la camera
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    while cam.isOpened():
        ret, frame = cam.read()

        #colorarea frame-ului din BGR in RGB
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False

        #detectarea modelului
        pose_model = pose.process(img)
        
        #colorarea frame-ului din RGB in BGR
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(img, pose_model.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow("Camera Feed", img)
        if cv2.waitKey(1) == ord('q'):
            break
            
    cam.release()
    cv2.destroyAllWindows()

In [3]:
def calcul_unghi_2_drepte(a, b, c, d):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    d = np.array(d)
    
    vect_dir_ab = np.subtract(b, a)
    vect_dir_cd = np.subtract(d, c)
    
    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_cd = np.linalg.norm(vect_dir_cd)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_cd) / (lungime_ab * lungime_cd))
    unghi = np.rad2deg(unghi)

    return unghi



In [4]:
def calcul_unghi_orizont(a, b):
    a = np.array(a)
    b = np.array(b)
    c = [0, 0.5, 0]
    d = [0.5, 0.5, 0]
    vect_dir_ab = np.subtract(b, a)
    vect_dir_cd = np.subtract(d, c)
    
    lungime_ab = np.linalg.norm(vect_dir_ab)
    lungime_cd = np.linalg.norm(vect_dir_cd)
    unghi = np.arccos(np.dot(vect_dir_ab, vect_dir_cd) / (lungime_ab * lungime_cd))
    unghi = np.rad2deg(unghi)

    return unghi

In [5]:
def prelucrare_img(nume_imagine, tip):
    global factor_scalare
    global pose_model
    
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
    
        img = cv2.imread(nume_imagine)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
    
        #detectarea modelului
        pose_model = pose.process(img)
            
        #colorarea frame-ului din RGB in BGR
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        factor_scalare = int(img.shape[0] / 720)
        if tip == 1:
            img = adaug_linii_suport(img)

        copie = img.copy()
        for lin in range(0, img.shape[1], 50 * factor_scalare):
            cv2.line(copie, (lin, 0), (lin, img.shape[0]), (200, 200, 200), 1)

        for lin in range(0, img.shape[0], 50 * factor_scalare):
            cv2.line(copie, (0, lin), (img.shape[1], lin), (200, 200, 200), 1)

        img = cv2.addWeighted(img, 0.8, copie, 0.2, 0)
        cv2.imwrite('img_prelucrata.jpg', img)
        img_al = afisare_aliniament(img, mijloace);
        cv2.imwrite('img_prelucrata_aliniament.jpg', img_al)

In [6]:
def adaug_text_front(img, coord_y, denumire, unghi):
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype("arial.ttf", 20 * factor_scalare)
    
    box = draw.textbbox((0, 0), denumire, font)
    lungime_text = box[2] - box[0]
    inaltime_text = box[3] - box[1]
    draw.rectangle([(8, coord_y - inaltime_text - 12 * factor_scalare), (8 + lungime_text + 2 * factor_scalare, coord_y - 8 * factor_scalare)], (255, 255, 255))
    draw.text((10, coord_y - 2*inaltime_text), denumire, (0, 0, 0), font)

    box = draw.textbbox((0, 0), unghi, font)
    lungime_text = box[2] - box[0]
    inaltime_text = box[3] - box[1]
    draw.rectangle([(img.shape[1] - 8 - lungime_text, coord_y - inaltime_text - 12 * factor_scalare), (img.shape[1] - 8, coord_y - 8 * factor_scalare)], (255, 255, 255))
    draw.text((img.shape[1] - 6 - lungime_text, coord_y - 2*inaltime_text), unghi, (0, 0, 0), font)

    img = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
    return img

In [7]:
def adaug_linii_suport(img):
    global mijloace
    
    try:
            puncte = pose_model.pose_landmarks.landmark
    except:
            pass
   
    
    puncte_linii_suport_dr = [mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE]
    puncte_linii_suport_st = [mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE]
    denumiri = ["Urechi", "Umeri", "Solduri", "Genunchi", "Glezne"]
    mijloace = []
    
    for (puncte_cheie_dr, puncte_cheie_st, denumire) in zip(puncte_linii_suport_dr, puncte_linii_suport_st, denumiri) :
        punct11 = (int(puncte[puncte_cheie_dr.value].x * img.shape[1]), int(puncte[puncte_cheie_dr.value].y * img.shape[0]), int(puncte[puncte_cheie_dr.value].z))
        punct12 = (int(puncte[puncte_cheie_st.value].x * img.shape[1]), int(puncte[puncte_cheie_st.value].y * img.shape[0]), int(puncte[puncte_cheie_dr.value].z))
        cv2.line(img, punct11[:-1], punct12[:-1], (255, 255, 255), factor_scalare)
        cv2.circle(img, punct11[:-1], 10, (0, 0, 0), factor_scalare)
        cv2.circle(img, punct12[:-1], 10, (0, 0, 0), factor_scalare)
        coord_y = int(((puncte[puncte_cheie_dr.value].y + puncte[puncte_cheie_st.value].y)/2) * img.shape[0])
        coord_x = int(((puncte[puncte_cheie_dr.value].x + puncte[puncte_cheie_st.value].x)/2) * img.shape[1])
        mijloace.append((coord_x, coord_y));
        punct1 = (0, coord_y)
        punct2 = (img.shape[1], coord_y)
        cv2.line(img, punct1, punct2, (123, 45, 200), factor_scalare)
        unghi = round(calcul_unghi_orizont(punct11, punct12), 2)
        unghi_str = str(unghi) + "°"
        img = adaug_text_front(img, coord_y, denumire, unghi_str)
       
        
    copie = img.copy()
    for i, puncte_cheie_dr in enumerate(puncte_linii_suport_dr[1:-1], 1):
            punct11 = (int(puncte[puncte_cheie_dr.value].x * img.shape[1]), int(puncte[puncte_cheie_dr.value].y * img.shape[0]))
            puncte_cheie_next = puncte_linii_suport_dr[i + 1]
            punct12 = (int(puncte[puncte_cheie_next.value].x * img.shape[1]), int(puncte[puncte_cheie_next.value].y * img.shape[0]))
            cv2.line(copie, punct11, punct12, (255, 255, 255), 2 * factor_scalare)
          
    for i, puncte_cheie_st in enumerate(puncte_linii_suport_st[1:-1], 1):
            punct11 = (int(puncte[puncte_cheie_st.value].x * img.shape[1]), int(puncte[puncte_cheie_st.value].y * img.shape[0]))
            puncte_cheie_next = puncte_linii_suport_st[i + 1]
            punct12 = (int(puncte[puncte_cheie_next.value].x * img.shape[1]), int(puncte[puncte_cheie_next.value].y * img.shape[0]))
            cv2.line(copie, punct11, punct12, (255, 255, 255), 2 * factor_scalare)

    img = cv2.addWeighted(img, 0.8, copie, 0.2, 0)

    return img

In [62]:
def afisare_aliniament(img, mijloace):
    copie_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(copie_pil)
    
    (_, y_urechi) = mijloace[0]
    (x_glezne, _) = mijloace[4]
    cv2.line(img, mijloace[4], (x_glezne, y_urechi), (0, 255, 0), factor_scalare)
    for i, (x, y) in enumerate(mijloace[:-1]):
        urm_x, urm_y = mijloace[i+1]
        pct_filler = [(x, y), (urm_x, urm_y), (x_glezne, urm_y), (x_glezne, y)]
        if (x > x_glezne and urm_x < x_glezne) or (x < x_glezne and urm_x > x_glezne):
            y_int = intersectie(x_glezne, y, x_glezne, urm_y, x, y, urm_x, urm_y)
            draw.polygon([(x, y), (x_glezne, y_int), (x_glezne, y)], (0, 255, 0))
            draw.polygon([(urm_x, urm_y), (x_glezne, y_int), (x_glezne, urm_y)], (0, 255, 0))
        else:
            draw.polygon(pct_filler, (0, 255, 0))
            
        cv2.line(img, (x, y), (urm_x, urm_y), (0, 0, 255), factor_scalare)
        cv2.circle(img, (x, y), 10, (0, 0, 0), factor_scalare)

    x, y = mijloace[-1]
    cv2.circle(img, (x, y), 10, (0, 0, 0), factor_scalare)
    copie = cv2.cvtColor(np.array(copie_pil), cv2.COLOR_RGB2BGR)
    img = cv2.addWeighted(img, 0.9, copie, 0.2, 0)
    
    return img

In [63]:
def intersectie(x11, y11, x12, y12, x21, y21, x22, y22):
    y_int = int(((x11 * y12 - y11 * x12) * (y21 - y22) - (y11 - y12) * (x21 * y22 - y21 * x22)) / ((x11 - x12) * (y21 - y22) - (y11 - y12) * (x21 - x22)))
    return y_int

In [65]:
prelucrare_img("img_test_2.jpeg", 1)

In [27]:
factor_scalare

2